---

### **Chapter 3-19**

#### **함수가 여러 값을 반환하는 경우 절대로 네 값 이상을 언패킹하지 말라.**



- 함수가 여러 값을 반환하기 위해 값들을 튜플에 넣어서 반환하고, 호출하는 쪽에서는 언패킹 구문 사용가능


In [2]:
def get_user_info():
    name = "Minje"
    age = 25
    location = "Seoul"
    return name, age, location  # (name, age, location) 튜플 반환

# 호출하는 쪽에서 언패킹
name, age, city = get_user_info()
print(f"{name}는 {city}에 거주 중.")

Minje는 Seoul에 거주 중.


- 함수가 반환한 여러 값을, 모든 값을 처리하는 별표식을 사용해 언패킹 가능.


In [3]:
def get_sensor_data():
    # 시간, 온도, 습도, 압력, 조도 등 여러 데이터 반환
    return "2026-02-12", 22.5, 45, 1013, 500

# 첫 번째 값(시간)만 받고 나머지는 리스트로 묶기
timestamp, *measurements = get_sensor_data()

print(timestamp)      # 2026-02-12
print(measurements)   # [22.5, 45, 1013, 500] (나머지 값들이 리스트로 저장됨)

2026-02-12
[22.5, 45, 1013, 500]


- 언패킹 구문에 변수가 4개 이상 나오면 실수하기 쉬우므로, 작은 클래스를 반환하거나 namedtuple 인스턴스 반환.

---

### **Chapter 3-20**

#### **None을 반환하기보다는 예외를 발생시켜라.**



- 특별한 의미를 표시하는 None을 반환하는 함수를 사용하면 None과 다른 값(Ex. 0이나 빈 문자열)이 조건문에서 False로 평가될 수 있기 때문에 실수하기 쉽다.


In [ ]:
# None 반환 시 발생할 수 있는 위험성
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

x, y = 0, 5
result = divide(x, y)

# result가 0인데, 파이썬 조건문에서 0은 False로 평가됨
if not result:
    print("잘못된 입력입니다.") # 실제로는 계산 결과가 0인 것뿐인데 에러로 처리됨!

잘못된 입력입니다.


- 특별한 상황을 표현하기 위해 None을 반환하는 대신 예외를 발생시켜라 


In [ ]:
# 해결책: 예외(Exception) 발생시키기
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        # None 대신 구체적인 예외를 발생시킴
        raise ValueError("입력값이 잘못되었습니다. 0으로 나눌 수 없습니다.") from e

x, y = 5, 0

try:
    result = divide(x, y)
except ValueError as e:
    print(f"에러 발생: {e}")
else:
    print(f"결과: {result}")

- 함수가 특별한 경우를 포함하는 그 어떤 경우에도 절대로 None을 반환하지 않는다는 사실을 타입 애너테이션으로 명시 가능.

In [ ]:
from typing import Optional

# [나쁜 예] Optional[float]은 float 또는 None이 반환될 수 있음을 의미
def divide_bad(a: float, b: float) -> Optional[float]:
    if b == 0:
        return None
    return a / b

# [좋은 예] 반환 타입을 명확히 float으로 고정
def divide_good(a: float, b: float) -> float:
    """나눗셈 수행. 0으로 나누면 ValueError 발생."""
    if b == 0:
        raise ValueError("Invalid inputs")
    return a / b


<class 'float'>
<class 'float'>


---

### **Chapter 3-21**

#### **변수 영역과 클로저의 상호작용 방식을 이해하라.**


- Closer함수는 자신이 정의된 영역 외부에서 정의된 변수도 참조할 수 있다.


In [18]:
def make_watcher():
    count = 100  # 감싸는 영역의 변수 (watch함수 기준 외부 변수)
    
    def watch():
        # 외부 변수 count를 참조할 수 있음
        print(f"현재 카운트: {count}")
        
    return watch

watcher = make_watcher()
watcher()  # 출력: 현재 카운트: 100

현재 카운트: 100


- 기본적으로 클로저 내부에 사용한 대입문은 클로저를 감싸는 영역에 끼칠 수 없음.


In [ ]:
def make_watcher():
    count = 0
    
    def update():
        count = 999  # 외부의 count를 바꾸는 게 아니라, 새로운 지역 변수 count를 생성함
        print(f"내부 count: {count}")
        
    return update

watcher = make_watcher()
watcher()  # 출력: 내부 count: 999
# 하지만 make_watcher의 count는 여전히 0입니다.

내부 count: 999


- nonlocal : 클로저가 자신을 감싸는 영역의 변수를 변경한다는 사실을 표시할 때
- 간단한 함수가 아닌 경우에는 non local문을 사용 비권장


In [20]:
def make_counter():
    count = 0
    
    def counter():
        nonlocal count  # "이 count는 내 지역 변수가 아니라 바깥쪽 꺼야"라고 선언
        count += 1
        return count
        
    return counter

my_counter = make_counter()
print(my_counter())  # 1
print(my_counter())  # 2

1
2


---

### **Chapter 3-22**

#### **변수 위치 인자를 사용해 시각적인 잡음을 줄여라**


- def문에서 `*args`를 사용하면 함수가 가변 위치 기반 인자를 받을 수 있다.


In [ ]:
def log_message(message, *args):
    if not args:
        print(message)
    else:
        # args는 (arg1, arg2, ...) 형태의 튜플
        values = ", ".join(str(x) for x in args)
        print(f"{message}: {values}")

log_message("데이터 확인", 10, 20, 30) 
# 출력: 데이터 확인: 10, 20, 30

데이터 확인: 10, 20, 30


- `*`연산자를 사용하면 가변 인자를 받는 함수에게 시퀀스 내의 모든 원소들을 전달 가능


In [22]:
def add(a, b, c):
    return a + b + c

numbers = [1, 2, 3]
# print(add(numbers))  # 에러: 인자 3개가 필요한데 리스트 1개만 전달됨
print(add(*numbers))   # 정상: add(1, 2, 3)으로 호출됨

6


- 제너레이터에 `*`연산자를 사용하면 프로그램이 메모리를 모두 소진하고 중단될 수 있음.
- `*args`를 받는 함수에 새로운 위치 기반 인자를 넣으면 감지하기 힘든 버그가 생길 수 있음.


In [23]:
# [기존 코드]
def calculate_total(price, *tax_rates):
    total = price
    for rate in tax_rates:
        total += price * rate
    return total

# 호출: calculate_total(1000, 0.1, 0.05) -> 정상

# [수정된 코드] 실수로 중간에 새로운 인자 'discount'를 추가함
def calculate_total(price, discount, *tax_rates):
    # 이제 기존 호출에서의 0.1이 'discount'로 들어가 버림!
    # 세금율 리스트인 tax_rates에는 0.05만 남게 됨
    ...

# 기존 호출 코드는 에러 없이 실행되지만, 결과값은 틀리게 나옴 (감지하기 힘든 버그)
print(calculate_total(1000, 0.1, 0.05))

None


---

### **Chapter 3-23**

#### **키워드 인자로 선택적인 기능을 제공하라**


- 함수를 호출할 때 값을 순서대로 넣을 수도(위치), 이름=값 형태로 넣을 수도(키워드) 있다.


In [ ]:
def calculate_velocity(distance, time):
    return distance / time

# 위치 인자: 순서가 중요함
print(calculate_velocity(100, 10))

# 키워드 인자: 순서가 바뀌어도 상관없음
print(calculate_velocity(time=10, distance=100))

- 인자가 많아질수록 각 숫자가 무엇을 의미하는지 알기 어렵다. 키워드를 사용하면 그 목적이 명확해짐.


In [ ]:
# [나쁜 예] 각 숫자가 무엇을 의미하는지 한눈에 알기 어려움
log_event("Error", True, 0, 10)

# [좋은 예] 각 인자의 역할이 명확함
log_event(
    message="Error",
    is_critical=True,
    retry_count=0,
    delay_seconds=10
)

- 이미 많은 곳에서 사용 중인 함수에 새로운 기능을 추가할 때, 디폴트 값을 가진 키워드 인자를 사용하면 기존 코드를 수정하지 않아도 된다.


In [ ]:
# [기존 함수]
def connect_server(address):
    print(f"{address}에 연결 시도")

# [업데이트 후] 새로운 기능(timeout)을 추가하되, 기본값을 설정
def connect_server(address, timeout=30): 
    print(f"{address}에 {timeout}초 동안 연결 시도")

# 기존 호출 코드: 수정 없이도 여전히 잘 동작함 (timeout은 30으로 적용)
connect_server("127.0.0.1")

# 새로운 기능이 필요한 곳에서만 사용
connect_server("127.0.0.1", timeout=5)

- 함수 정의에서 필수적이지 않은 옵션들은 위치 인자로 전달하기보다, 항상 키워드를 명시해서 전달하는 것이 권장됨


In [ ]:
def resize_image(data, width, height, shadow=False, grayscale=False):
    # 이미지 처리 로직
    pass

# [위험한 호출] 나중에 shadow와 grayscale의 순서가 바뀌면 버그 발생
resize_image(img_data, 200, 200, True, False)

# [안전한 호출] 선택적 인자는 키워드를 사용하여 명확히 전달
resize_image(img_data, 200, 200, shadow=True, grayscale=False)

---

### **Chapter 3-24**

#### **None과 독스트링을 사용해 동적인 디폴트 인자를 지정하라**


- 파이썬에서 함수의 디폴트 값은 함수가 정의되는 시점(모듈 로드 시점)에 딱 한 번만 평가. 
    - 리스트[]나 딕셔너리{} 같은 가변 객체를 디폴트 값으로 지정하면, 모든 함수 호출이 동일한 객체를 공유하게 됩


In [ ]:
from datetime import datetime
import time

# [위험한 예시] 
def log_event(message, when=datetime.now()):  # datetime.now()가 처음 사용될때만 정의됨.
    print(f"{when}: {message}")

log_event("첫 번째 이벤트")
time.sleep(10)
log_event("두 번째 이벤트") # 첫 번째와 똑같은 시간이 출력됨!

2026-02-12 14:21:51.648140: 첫 번째 이벤트
2026-02-12 14:21:51.648140: 두 번째 이벤트


- 동적인 값을 가질 수 있는 키워드 인자의 디폴트 값을 표현할 때는  None을 사용
    - 그리고 함수의 독스트링에 실제 동적인 디폴트 인자가 어떻게 동작하는지 문서화


In [27]:
def log_event(message, when=None):
    """이벤트를 로그에 남깁니다.

    Args:
        message: 로그에 남길 메시지.
        when: 이벤트가 발생한 시각(datetime).
            기본값은 함수가 호출되는 시점의 현재 시각입니다.
    """
    if when is None:
        when = datetime.now()
    print(f"{when}: {message}")

log_event("정상적인 로그")
time.sleep(5)
log_event("새로운 시각의 로그") # 이제 호출 시점의 시간이 정확히 반영됨

2026-02-12 14:23:14.413975: 정상적인 로그
2026-02-12 14:23:19.414166: 새로운 시각의 로그


- 타입 애너테이션을 사용할 때도 None 을 사용해 키워드 인자의 디폴트 값을 표현하는 방식 적용 가능.

In [28]:
from typing import Optional, List

def add_item(item: str, items: Optional[List[str]] = None) -> List[str]:
    """리스트에 아이템을 추가합니다.
    
    Args:
        item: 추가할 문자열.
        items: 추가 대상 리스트. None이면 새 리스트를 생성합니다.
    """
    if items is None:
        items = []
    items.append(item)
    return items

# 이제 매번 새로운 리스트가 보장됩니다.
print(add_item("파이썬")) # ["파이썬"]
print(add_item("자바"))   # ["자바"]

['파이썬']
['자바']


`함수 정의 시점 vs 호출 시점: 디폴트 값은 정의 시점에 정해집니다. datetime.now()나 []는 정의할 때의 값으로 고정됨`

---

### **Chapter 3-25**

#### **위치로만 인자를 지정하게 하거나 키워드로만 인자를 지정하게 해서 함수 호출을 명확하게 만들라**


- 키워드로만 지정해야하는 인자를 사용하면 호출하는 쪽에서 특정 인자를 반드시 키워드를 사용해 호출하도록 강제 가능.
    - 키워드로만 지정해야하는 인자는 인자 목록에서 * 다음 위치


In [31]:
def safe_division(number, divisor, *, ignore_overflow=False, ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        raise

# [나쁜 호출] 에러 발생 (위치 기반으로 전달 불가)
# safe_division(1, 0, False, True) 

# [좋은 호출] 반드시 이름을 명시해야 함
result = safe_division(1, 0, ignore_zero_division=True)
print(result) # inf


inf


- 위치로만 지정해야하는 인자를 사용하면 호출하는 쪽에서 키워들르 사용해 인자를 지정하지 못하게 만들 수 있고, 이에 따라 함수 구현과 함수 호출 지점 사이의 결합을 줄일 수 있음
    - 위치로만 지정해야하는 인자는 인자 목록에서 / 앞에 위치


In [29]:
def format_data(data, /, prefix=""):
    return f"{prefix}{data}"

# [정상 호출] 위치 기반으로만 전달 가능
print(format_data("Hello", "LOG: "))

# [에러 발생] 'data'라는 이름을 키워드로 사용할 수 없음
# print(format_data(data="Hello", prefix="LOG: "))

LOG: Hello


- 인자목록에서 `/`와 `*` 사이에 있는 파라미터는 키워드를 사용해 전달해도 되고 위치를 기반으로 전달해도 됨.



In [32]:
# a, b: 위치 전용 (Position-only)
# c: 위치 또는 키워드 가능 (Standard)
# d: 키워드 전용 (Keyword-only)
def complex_func(a, b, /, c, *, d):
    print(f"a={a}, b={b}, c={c}, d={d}")

# 올바른 호출 예시
complex_func(1, 2, 3, d=4)          # c를 위치로 전달
complex_func(1, 2, c=3, d=4)        # c를 키워드로 전달

# 잘못된 호출 예시
# complex_func(a=1, b=2, c=3, d=4)  # 에러: a, b는 키워드 사용 불가
# complex_func(1, 2, 3, 4)          # 에러: d는 반드시 키워드 사용 필요

a=1, b=2, c=3, d=4
a=1, b=2, c=3, d=4


---

### **Chapter 3-26**

#### **functools.wrap을 사용해 함수 데코레이터를 정의하라**


- 데코레이터는 실행 시점에 함수가 다른 함수를 변경할 수 있게 해주는 구문.


In [36]:
# 데코레이터는 함수를 인자로 받아 새로운 함수를 반환하는 '래퍼(Wrapper)' 역할
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f"{func.__name__}({args!r}, {kwargs!r}) -> {result!r}")
        return result
    return wrapper

@trace
def add(a, b):
    """두 수를 더한 값을 반환함."""
    return a + b

add(1, 2) # 출력: add((1, 2), {}) -> 3

add((1, 2), {}) -> 3


3

- 데코레이터를 사용하면 디버거 등 인트로스펙션을 사용하는 도구가 잘못 작동할 수 있다.

- 인트로스펙션이란? 
    - 실행 시점에 프로그램이 자기 자신의 객체 정보를 조사하는 것을 말합니다. 디버거나 자동 문서화 도구가 이 기능을 사용하는데, 위처럼 이름이 바뀌어버리면 도구들이 길을 잃게 됩니다.


In [34]:
print(add.__name__) # 출력: wrapper (add가 아님!)
print(add.__doc__)  # 출력: None (원래 독스트링이 사라짐)

# 도움말을 확인해도 wrapper 함수 정보만 나옵니다.
# help(add)

wrapper
None


- 직접 데코레이터를 구현할 때 인트로스펙션에서 문제가 생기지 않길 바란다면 functools의 wraps 데코레이터를 사용하라
    - wraps는 데코레이터 내부의 래퍼 함수가 원래 함수의 메타데이터를 그대로 복사해오도록 돕는 데코레이터용 데코레이터

In [35]:
from functools import wraps

def trace(func):
    @wraps(func)  # 중요: func의 메타데이터를 wrapper로 복사함
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f"{func.__name__}({args!r}, {kwargs!r}) -> {result!r}")
        return result
    return wrapper

@trace
def add(a, b):
    """두 수를 더한 값을 반환함."""
    return a + b

# 이제 원래 함수의 정보가 잘 보존됩니다!
print(add.__name__) # 출력: add
print(add.__doc__)  # 출력: 두 수를 더한 값을 반환함.

add
두 수를 더한 값을 반환함.


In [37]:
import time
from functools import wraps

def timer(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()  # [전] 시작 시간 기록
        
        result = func(*args, **kwargs)  # [중] 함수 실행
        
        end = time.time()    # [후] 끝난 시간 기록
        print(f"⏱️ {func.__name__} 함수 실행 시간: {end - start:.4f}초")
        return result
    return wrapper

@timer
def heavy_computation():
    """아주 무거운 계산을 하는 함수"""
    time.sleep(1.5) # 1.5초 대기
    return "계산 완료"

print(heavy_computation())

⏱️ heavy_computation 함수 실행 시간: 1.5001초
계산 완료
